In [24]:
import numpy as np
import pandas as pd

In [25]:
data = pd.read_excel('p2_1c.xlsx')

In [26]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]
x_train = X.iloc[0:90,:].values
x_test = X.iloc[90:,:].values
y_train = Y[0:90].values.reshape(-1,1)
y_test = Y[90:].values.reshape(-1,1)

In [27]:
class Node():
    def __init__(self, attribute=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.attribute = attribute
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [28]:
class DecisionTree():
    def __init__(self, min_sample=2, max_depth=8):
        self.root = None
        self.min_sample = min_sample
        self.max_depth = max_depth
        
    def build_tree(self, dataset, depth=0):        
        X, Y = dataset[:,0:3], dataset[:,3]
        n_rows, n_cols = np.shape(X)
        if n_rows>=self.min_sample and depth<=self.max_depth:
            best_split = self.get_best_split(dataset, n_rows, n_cols)
            if best_split["info_gain"]>0:
                lefttree = self.build_tree(best_split["left"], depth+1)
                righttree = self.build_tree(best_split["right"], depth+1)
                return Node(best_split["attribute"], best_split["threshold"], 
                            lefttree, righttree, best_split["info_gain"])
        leaf_value = self.calculate_node(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, n_rows, n_cols):
        best_split = {}
        info_gain = -float('inf')
        for col in range(n_cols):
            values = dataset[:, col]
            _thresholds = []
            for i in range(len(values)):
                if i != 0:
                    current_value = values[i]
                    next_value = values[i - 1]
                    split_value = (current_value + next_value) / 2
                    _thresholds.append(split_value)
            for threshold in _thresholds:
                left, right = self.split(dataset, col, threshold)
                if len(left)>0 and len(right)>0:
                    y, left_y, right_y = dataset[:, 0:3], left[:, 0:3], right[:, 0:3]
                    IG = self.information_gain(y, left_y, right_y)
                    if IG>info_gain:
                        best_split["attribute"] = col
                        best_split["threshold"] = threshold
                        best_split["left"] = left
                        best_split["right"] = right
                        best_split["info_gain"] = IG
                        info_gain = IG 
            _thresholds = []
        return best_split
    
    def split(self, dataset, col, threshold):
        dataset_left = np.array([i for i in dataset if i[col]<=threshold])
        dataset_right = np.array([i for i in dataset if i[col]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, data, l_child, r_child):
        total = len(l_child) + len(r_child)
        prob_left_child = len(l_child) / total
        prob_right_child = len(r_child) / total
        entropy_left_child = self.entropy(l_child)
        entropy_right_child = self.entropy(r_child)
        entropy_of_split_data = (prob_left_child * entropy_left_child) + (prob_right_child * entropy_right_child)
        gain = self.entropy(data) - entropy_of_split_data
        return gain
    
    def entropy(self, y):
        labels = np.unique(y)
        entropy = 0
        for attribute in labels:
            p_attribute = len(y[y == attribute]) / len(y)
            entropy += -p_attribute * np.log2(p_attribute)
        return entropy

    def calculate_node(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        preditions = [self.classify(x, self.root) for x in X]
        return preditions
    
    def classify(self, x, tree):
        if tree.value!=None: return tree.value
        values = x[tree.attribute]
        if values<=tree.threshold:
            return self.classify(x, tree.left)
        else:
            return self.classify(x, tree.right)


In [29]:
def accuracy(y_test, y_pred):
    correct = 0
    for i in range(len(y_test)):
        if y_test[i] == y_pred[i]:
            correct += 1
    return correct/float(len(y_test)) * 100.0    

In [30]:
for i in range(1, 9):
    print('Depth: ', i)
    classifier = DecisionTree(min_sample=2, max_depth=i)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_train)
    acc = accuracy(y_train, y_pred)
    print('Accuracy: ', acc)

Depth:  1
Accuracy:  61.111111111111114
Depth:  2
Accuracy:  61.111111111111114
Depth:  3
Accuracy:  71.11111111111111
Depth:  4
Accuracy:  75.55555555555556
Depth:  5
Accuracy:  84.44444444444444
Depth:  6
Accuracy:  100.0
Depth:  7
Accuracy:  100.0
Depth:  8
Accuracy:  100.0


In [31]:
for i in range(1, 9):
    print('Depth: ', i)
    classifier = DecisionTree(min_sample=2, max_depth=i)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    acc = accuracy(y_test, y_pred)
    print('Accuracy: ', acc)

Depth:  1
Accuracy:  16.666666666666664
Depth:  2
Accuracy:  20.0
Depth:  3
Accuracy:  40.0
Depth:  4
Accuracy:  56.666666666666664
Depth:  5
Accuracy:  40.0
Depth:  6
Accuracy:  40.0
Depth:  7
Accuracy:  40.0
Depth:  8
Accuracy:  40.0


The accuracy on the train data increases till 100% as depth increases.
for test data the best accuracy is at depth 4, 56 % after that it starts reducing. Therefore depth 4 onwards the algorithm shows overfitting.